# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
  ※この問題は仕様として **`id` 昇順で返す** ため、最終的に `ORDER BY id` を付けます
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は **ID 基準**、表示は仕様どおり

## 1) 問題

* `連番 ID を持つ Seat テーブルで、学生の席（id）を 2 人ずつ入れ替える。人数が奇数なら最後の 1 人はそのまま。結果は id 昇順で返す。`
* 入力: `Seat(id INT PRIMARY KEY, student TEXT)`
* 出力: `id, student`

  * 奇数 id は「次が存在するときだけ」`id+1` にスワップ
  * 偶数 id は常に `id-1` にスワップ
  * 最後の孤立した奇数 id は据え置き
  * **最終出力は id 昇順**

## 2) 最適解（単一クエリ）

> PostgreSQL では **CTE + ウィンドウ**で全体最大 `id` を取り、**ビット演算**で LSB をトグルするのが簡潔かつ高速です。
> 0 始まりに直して LSB を XOR で切り替えることで `1↔2, 3↔4, ...` の対応を **`((id-1) # 1) + 1`** の 1 式で表現できます（`#` は XOR）。末尾が奇数の 1 件だけ据え置きます。

```sql
WITH win AS (
  SELECT id, student, MAX(id) OVER () AS max_id
  FROM seat
)
SELECT
  CASE
    WHEN (id & 1) = 1 AND id = max_id    -- 最後の孤立奇数は据え置き
      THEN id
    ELSE ((id - 1) # 1) + 1              -- それ以外は 1↔2, 3↔4 を一発変換
  END AS id,
  student
FROM win
ORDER BY id;                              -- 仕様：id 昇順

Runtime 179 ms
Beats 70.12%

```

### 代替（CTE不要・定数サブクエリ）

> ウィンドウを避け、`MAX(id)` を 1 回だけ取得して全行に結合。`% 2` より **`& 1`** の方が軽量です。

```sql
SELECT
  CASE
    WHEN (s.id & 1) = 1 AND s.id = m.max_id THEN s.id
    ELSE ((s.id - 1) # 1) + 1
  END AS id,
  s.student
FROM seat AS s
CROSS JOIN (SELECT MAX(id) AS max_id FROM seat) AS m
ORDER BY id;

Runtime 188 ms
Beats 49.06%

```

## 3) 要点解説

* **発想**: 「2 人 1 組の入替」は **最下位ビットのトグル**で表現できる

  * 0 始まりに直す → `id-1`
  * LSB を XOR で切替 → `(id-1) # 1`
  * 1 始まりに戻す → `+1`
* **末尾の孤立奇数**のみ例外なので、`MAX(id)` と `(id & 1)` の組み合わせでガード
* **演算の軽量化**: `MOD(id,2)` や `id % 2` より **`id & 1`** の方が軽い傾向
* **並び順**: 問題仕様が「id 昇順」なので、最終 `ORDER BY id` を明示

## 4) 計算量（概算）

* フルスキャン主体で **O(N)**

  * `MAX(id) OVER ()` は 1 パス集約の伝播
  * 代替版の `CROSS JOIN (SELECT MAX(id))` も 1 回評価の定数派生表
* ビット演算は定数時間で、分岐は 1 回のみ

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Seat 入力] --> B["MAX(id) を取得"]
  B --> C["LSB トグルで 1↔2, 3↔4"]
  C --> D[最後の孤立奇数を据え置き]
  D --> E[id 昇順で整列]
  E --> F[出力 id, student]
```

この問題は**最終的に「並べ替えが必須」**（新しい `id` は元の `id` と同一順序にならない）なので、どんな書き方でも最終段で **`ORDER BY` によるソートコスト O(N log N)** は避けられません。したがって 179ms はかなり健闘値。ここから伸ばせるとすれば「微々たるマイクロ最適化」と「無駄な再計算を減らす」くらいです。

以下、改善の余地とおすすめ最終形です。

---

## ちょい改善ポイント

1. **偶奇判定の再計算を避ける**

* `(id & 1)` を何度も評価しないように、サブクエリで一度だけ求めて使い回し。

2. **ガード条件を最小化**

* 例外は「`max_id` が奇数かつ `id = max_id`」の 1 ケースのみ。
  → それ以外は XOR で一発変換：`((id - 1) # 1) + 1`

3. **`MAX(id)` は 1 パスで全体に配る**

* PG 16 ならウィンドウ `MAX(id) OVER ()` は十分速いです（CTE はデフォでインライン化）。
  実測でも CTE 版が速いのでそのままでOK。

---

## 推奨クエリ（最終形：分岐・再計算を最小化）

```sql
WITH s AS (
  SELECT
    id,
    student,
    (id & 1) AS odd,          -- 偶奇を一度だけ計算
    MAX(id) OVER () AS max_id -- 全体最大を 1 パスで配る
  FROM seat
)
SELECT
  CASE
    WHEN odd = 1 AND id = max_id       -- 末尾の孤立奇数だけ据え置き
      THEN id
    ELSE ((id - 1) # 1) + 1            -- それ以外は 1↔2, 3↔4 を一発変換
  END AS id,
  student
FROM s
ORDER BY id;                            -- 問題仕様：id 昇順

Runtime 180 ms
Beats 67.29%

```

### 期待

* あなたの **179ms** と同等か、わずかに良化する可能性。
* これ以上は並べ替えコストがボトルネックなので、**劇的改善は見込みづらい**です。

---

## 他の選択肢（状況依存で僅差）

* **定数サブクエリ JOIN 版**（`CROSS JOIN (SELECT MAX(id))`）
  環境次第で CTE より速いこともありますが、あなたの計測では CTE 版が優勢。維持でOK。
* **`UNION ALL` で 3 区分**（奇数・偶数・末尾奇数）
  結局、最終の `ORDER BY id` は必要なので、多くのケースで優位にはなりません。

---

## まとめ

* この課題は**並べ替えが支配的**。今の 179ms は上出来です。
* 上の最終形（偶奇の事前計算＋XOR一発＋最小ガード）にしておけば、**読みやすさと軽さのバランス**が最良クラス。
* さらに詰める余地は実運用なら「ワークメモリや並列度（`work_mem`/`max_parallel_workers_per_gather`）」ですが、オンライン判定環境では調整不可なので、SQLの形としてはここが完成形です。

